In [34]:
import pandas as pd
import numpy as np
from typing import List
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

In [77]:
menu_df = pd.read_csv("Menus/menu1.csv")

In [79]:
documents = [
    Document(
        page_content=f"{row['name']}. {row['description']}. Category: {row['category']}.",
        metadata={
            "id": row["id"],
            "description": row["description"],
            "name": row["name"],
            "price": row["price"],
            "category": row["category"],
            "tags": row["tags"]
        }
    )
    for _, row in menu_df.iterrows()
]

embedding_model_nomic = OllamaEmbeddings(model = 'nomic-embed-text')
menu_vector_store = FAISS.from_documents(documents, embedding_model_nomic)

In [134]:
def semantic_filter(
    query: str,
    vector_store: FAISS,
    embedding_model: OllamaEmbeddings,
    threshold: float = 0.75,
    k: int = 10
    ) -> List[Document]:

    """
    Perform a semantic search and filter by similarity threshold.

    Returns a list of tuples: (menu item name, similarity score, page content).
    """
    
    # Step 1: Embed query
    query_vector = embedding_model.embed_query(query)

    # Step 2: Search using FAISS
    D, I = vector_store.index.search(np.array([query_vector]), k=k)
    similarities = 1 - D[0]  # Convert distance to similarity

    # Step 3: Filter results above threshold
    results = []
    for i, sim in zip(I[0], similarities):
        if sim >= threshold:
            doc_id = vector_store.index_to_docstore_id[i]
            doc = vector_store.docstore._dict[doc_id]
            results.append(doc)

    return results

In [145]:
def search_menu(user_query, category_filter=None, tags_filter=None):

    # Semantic similarity search
    if user_query:
        results = semantic_filter(
            query=user_query,
            vector_store=menu_vector_store,
            embedding_model=embedding_model_nomic,
            threshold=0.1,
            k=10
        )
    else:
    # If no query, return all documents
        results = list(menu_vector_store.docstore._dict.values())


    if category_filter:
        results = [doc for doc in results if doc.metadata["category"].lower() == category_filter.lower()]

    if tags_filter:
        results = [doc for doc in results if tags_filter.lower() in [tag.lower() for tag in doc.metadata["tags"].split(",")]]

    return results

In [152]:
response = search_menu(user_query =None, category_filter="Appetizer", tags_filter=None)  # Example usage
for i in [r.metadata['name']+'\t'+str(r.metadata['price'])+'\n'+r.metadata['description']+'\n' for r in response]:  # Print names of the filtered results
    print(i)

Grace Garden Salad	6.5
Mixed greens with vinaigrette dressing and croutons

Daily Bread Basket	4.99
Assorted artisan breads with olive oil and balsamic

Savior’s Samosas	5.99
Golden pastry filled with spiced potatoes and peas

Sanctified Spring Rolls	6.0
Rice paper rolls with tofu, mint, and vermicelli

Samosa Chaat	6.99
Crushed samosas topped with yogurt, chutneys, and spices

Spring Rolls	5.75
Crispy rolls filled with vegetables and noodles

Gobi Manchurian	7.5
Crispy cauliflower tossed in tangy Indo-Chinese sauce

Vegetable Momos	6.5
Steamed dumplings filled with mixed vegetables

Hot and Sour Soup	4.99
Spicy and tangy Chinese-style soup

Fried Wontons	5.25
Deep-fried wontons filled with veggies or meat

